In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NeuralNet(nn.Module):
    """
    """
    def __init__(self,input_size=1000,hidden_size=100,output_size=10):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size, bias=True)
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size, bias=True)
    def forward(self,X):
        FX = F.relu(self.fc1(X)) # hidden layer activation features
        prob = F.softmax(self.fc2(FX),dim=1) # probability output
        return FX,prob


class DGDE:
    """
    The batch size per iteration is batch_size * num_class * num_domain
    """
    def __init__(self,input_size=1000,hidden_size=100,output_size=10,seed=1000,device=torch.device('cpu'),
                 epoch=200,lamda=1e-3,gamma=100,sigma=None,batch_size=4,lr=1e-3,log=True):
        args_values = locals()
        args_values.pop("self")
        for arg,value in args_values.items():
            setattr(self,arg,value)
            
    def fit(self,X_list,y_list,Xt,yt):
        torch.manual_seed(self.seed)
        net = NeuralNet(input_size=self.input_size,hidden_size=self.hidden_size,output_size=self.output_size).to(self.device)
        optimizer = optim.SGD(params=net.parameters(),lr=self.lr,momentum=0.9)
        
        for epoch in range(self.epoch):
            dataset_loaders, l = [], 0
            for Xs,ys in zip(X_list,y_list):
                for i,counts in zip(*np.unique(ys,return_counts=True)):
                    dataset = np.hstack((Xs[ys==i],ys[ys==i][:,None],l * np.ones((counts,1))))
                    dataset_loaders.append(torch.utils.data.DataLoader(dataset=torch.tensor(dataset),batch_size=self.batch_size,shuffle=True,drop_last=True))
                l = l + 1

            train_err,m_train = 0.0, 0.0

            for batches in zip(*dataset_loaders):
                Xyl = torch.cat(batches,dim=0)
                X,y,l = Xyl[:,:-2].to(self.device,torch.float32),Xyl[:,-2].to(self.device,torch.int64),Xyl[:,-1].to(self.device,torch.int64)

                m,domain_label = X.shape[0], torch.unique(l)
                FX,prob = net(X)

                num_class = prob.shape[1]
                negative_log_loss = -torch.mean(torch.sum(torch.log(prob) * F.one_hot(y,num_class),dim=1))        

                #=============KLD calculation===========
                if self.sigma is None:
                    pairwise_dist = torch.cdist(X,X,p=2)**2 
                    self.sigma = torch.median(pairwise_dist[pairwise_dist!=0])

                Delta = torch.as_tensor(y[:,None]==y,dtype=torch.float64,device=self.device) # construct the label kernel matrix
                FX_norm = torch.sum(FX ** 2, axis = -1)
                # construct the Gaussian kernel matrix
                #https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python
                K = torch.exp(-(FX_norm[:,None] + FX_norm[None,:] - 2 * torch.matmul(FX, FX.t())) / self.sigma)
                P = K * Delta
                invM = torch.inverse(1./ m * torch.matmul(P.t(),P) + self.lamda * torch.eye(m,device=self.device))

                alpha = torch.stack([torch.matmul(invM,1. / m * torch.sum(P[l==i],axis=0)) for i in domain_label],dim=1)
                Palpha = torch.clamp(torch.matmul(P,alpha),min=1e-8)
                ProbM = Palpha / Palpha.sum(axis=1,keepdim=True)

                KLD = 0.
                for i in domain_label:
                    KLD = KLD + torch.mean(torch.log(ProbM[l==i][:,i]))
                #======================================

                loss = negative_log_loss + self.gamma * KLD
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_err += loss.item() * m
                m_train += m

            with torch.no_grad():
                Xt,yt = torch.as_tensor(Xt,dtype=torch.float32,device=self.device),torch.as_tensor(yt,dtype=torch.int64,device=self.device)    
                pred = torch.argmax(net(Xt)[1],dim=1)
                correct = torch.sum((pred == yt)).item()
                m_test = len(yt)
            
            if True == self.log:
                print('epoch ',epoch,', training error ',train_err / m_train,', test acc. ',(correct / m_test) * 100) 
        self.net = net
        
    def score(self,Xt,yt):
        with torch.no_grad():
            Xt,yt = torch.as_tensor(Xt,dtype=torch.float32,device=self.device),torch.as_tensor(yt,dtype=torch.int64,device=self.device)    
            pred = torch.argmax(self.net(Xt)[1],dim=1)
            correct = torch.sum((pred == yt)).item()
            m_test = len(yt)
        return (correct / m_test) * 100

In [2]:
import numpy as np
import pandas as pd
import scipy.io as sio
import numpy.linalg as la
from sklearn.preprocessing import scale,LabelEncoder

In [3]:
def readData(tg,domains):
    data = sio.loadmat('ILS_PIE/' + tg + '.mat')
    Xt,yt = data['features'].astype(np.float64).T,data['labels'].ravel()
    yt = LabelEncoder().fit(yt).transform(yt).astype(np.float64)
    #Xt = Xt / la.norm(Xt,axis=1,keepdims=True)
    #Xt = scale(Xt)
    Xt = scale(Xt / Xt.sum(axis=1,keepdims=True))
    
    Xs_list,ys_list = [],[]
    for sc in domains:
        if sc != tg:
            data = sio.loadmat('ILS_PIE/' + sc + '.mat')
            Xs,ys = data['features'].astype(np.float64).T,data['labels'].ravel()
            ys = LabelEncoder().fit(ys).transform(ys).astype(np.float64)
            #Xs = Xs / la.norm(Xs,axis=1,keepdims=True)
            #Xs = scale(Xs)
            Xs = scale(Xs / Xs.sum(axis=1,keepdims=True))
            Xs_list.append(Xs),ys_list.append(ys)        
    
    return Xs_list,ys_list,Xt,yt

domains = ['C27', 'C09', 'C05', 'C37', 'C25', 'C02']

In [4]:
DEVICE = torch.device('cuda:0')
domcouples = []
res = []

for tg in domains:
    X_list,y_list,Xt,yt = readData(tg,domains)
    
    dgde = DGDE(input_size=1024,hidden_size=512,output_size=67,seed=0,device=DEVICE,
                         epoch=300,lamda=1e-2,gamma=100,sigma=None,batch_size=5,lr=1e-2,log=False)
    dgde.fit(X_list,y_list,Xt,yt)
    
    domcouples.append(tg)
    res.append(dgde.score(Xt,yt))
result = pd.DataFrame(data=res,index=domcouples, columns=['DGDE'])
print(result.values.mean())
result

84.54906987245295


,DGDE
C27,99.145299
C09,83.795309
C05,96.233120
C37,88.603989
C25,66.666667
C02,72.850036
